In [ ]:
### Load Essential Packages ###
import os

### Create New Directories for PNG Files ###
# traverse root directory, and list directories as dirs and files as files
for root, dirs, files in os.walk('/media/john/Hollander4TB/MESA_CT_Exam5/'):
    path = root.split(os.sep)
    path2 = path[-1]
    path3 = ''.join(path2)
    os.makedirs('/media/john/Hollander4TB/Pectoralis/Axial/' + path3, exist_ok=True)

In [ ]:
#####WINDOWING#####

###head and neck
#brain W:80 L:40
#subdural W:130-300 L:50-100
#stroke W:8 L:32 or W:40 L:40 3
#temporal bones W:2800 L:600 or W:4000 L:700
#soft tissues: W:350–400 L:20–60 4

###chest
#lungs W:1500 L:-600
#mediastinum W:350 L:50

###abdomen
#soft tissues W:400 L:50
#liver W:150 L:30

###spine
#soft tissues W:250 L:50
#bone W:1800 L:400


#####Path Sorting####
###axial = 10:-4
###coronal = 12:-4
###sagittal = 13:-4

import pydicom
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import sys
import glob
import csv
import cv2
import os
from os import listdir
from os.path import isfile, join

folder = '/media/john/Hollander4TB/MESA_CT_Exam5'
sub_folders = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]
sortedfolder = sorted(sub_folders)

for q in sortedfolder:
    files = []
    
    mypath = '/media/john/Hollander4TB/MESA_CT_Exam5/' + q + '/'
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    selectedfiles = [x[0:7] for x in onlyfiles]

    def most_frequent(List):
        return max(set(List), key = List.count)
    common = most_frequent(selectedfiles)

    ### Automatically Create PNG from DICOM for All Patients in Directory ###
    for filename in sorted(glob.glob('/media/john/Hollander4TB/MESA_CT_Exam5/' + q + '/' + common + '*.dcm')):
        files.append(pydicom.dcmread(filename))

    print("file count: {}".format(len(files)))

    # skip files with no SliceLocation (eg scout views)
    slices = []
    skipcount = 0
    for f in files:
        if hasattr(f, 'SliceLocation'):
            slices.append(f)
        else:
            skipcount = skipcount + 1

    print("skipped, no SliceLocation: {}".format(skipcount))

    # ensure they are in the correct order
    slices = sorted(slices, key=lambda s: s.SliceLocation)

    # pixel spacing and slice thickness
    ps1 = []
    ps2 = []
    ss = []
    man = []
    model = []
    imagetype = []

    # create 3D array
    img_shape = list(slices[0].pixel_array.shape)
    img_shape.append(len(slices))
    img3d = np.zeros(img_shape)

    # define windowing
    ww=1800
    wl=400
    U = 255
    W = U / ww
    b = (-U/ww) * (wl-ww/2)    
    
    
    try:
        # fill 3D array with the images from the files
        m = 0
        for i, s in enumerate(slices):
            img = s.pixel_array
            intercept = float(s.RescaleIntercept)
            slope = float(s.RescaleSlope)
            img2d = slope * img + intercept
            axial = img2d
            axial = W * axial + b
            axial = np.where(axial > U, U, axial)
            axial = np.where(axial < 0, 0, axial)
            axial -= axial.mean()
            axial /= (axial.std() + 1e-10)
            axial -= axial.min()
            axial = (255 * axial/np.max(axial)).astype('uint8')
            axial2 = cv2.resize(axial, (512, 512), interpolation = cv2.INTER_NEAREST) 
            cv2.imwrite('/media/john/Hollander4TB/vBMD/3D_Volume/' + q + '/axial_' + common + '_' + str(m) + '.png', axial2)
            if img2d.shape[0] == 512 and img2d.shape[1] == 512:
                img3d[:, :, i] = img2d
                m += 1
            else:
                m += 1
                continue

        ps1, ps2 = slices[0].PixelSpacing
        ss = slices[0].SliceThickness
        man = slices[0].Manufacturer
        model = slices[0].ManufacturerModelName
        imagetype = slices[0].ImageType
        fields = [q, common, ps1, ps2, ss, man, model, imagetype]
        with open('/media/john/Hollander4TB/vBMD/Info_vBMD.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow(fields)

        m = 0
        shapelist = [img3d.shape[0] , img3d.shape[1] , img3d.shape[2]]
        while m < min(shapelist):
            sagittal = img3d[:, m, :]
            sagittal = W * sagittal + b
            sagittal = np.where(sagittal > U, U, sagittal)
            sagittal = np.where(sagittal < 0, 0, sagittal)
            sagittal -= sagittal.mean()
            sagittal /= (sagittal.std() + 1e-10)
            sagittal -= sagittal.min()
            sagittal = (255 * sagittal/np.max(sagittal)).astype('uint8')
            sagittal2 = cv2.resize(sagittal, (640, 512), interpolation = cv2.INTER_NEAREST)
            sagittal3 = cv2.rotate(sagittal2, cv2.ROTATE_90_COUNTERCLOCKWISE)
            cv2.imwrite('/media/john/Hollander4TB/vBMD/3D_Volume/' + q + '/sagittal_' + common + '_' + str(m) + '.png', sagittal3)
            coronal = img3d[m, :, :].T
            coronal = W * coronal + b
            coronal = np.where(coronal > U, U, coronal)
            coronal = np.where(coronal < 0, 0, coronal)
            coronal -= coronal.mean()
            coronal /= (coronal.std() + 1e-10)
            coronal -= coronal.min()
            coronal = (255 * coronal/np.max(coronal)).astype('uint8')
            coronal2 = cv2.resize(coronal, (512, 640), interpolation = cv2.INTER_NEAREST)
            cv2.imwrite('/media/john/Hollander4TB/vBMD/3D_Volume/' + q + '/coronal_' + common + '_' + str(m) + '.png', coronal2)
        
            m += 1
    except TypeError:
        pass

In [ ]:
### Load Essential Packages ###
import numpy as np
import pandas as pd
import os
import cv2
from imutils import paths
import argparse
import glob
from glob import iglob

t1 = []
n_white_pix = []
n_white_pix2 = []
n_white_pix3 = []
names = []
names2 = []
index = 0

###Axial###
for filename in sorted(glob.glob('/home/john/Documents/MESArthritis/Ax_Cor_Sag/**/axial_*.png'), key=lambda name: (name[45:56], int(os.path.basename(name)[14:-4]))):
    image = cv2.imread(filename)
    image2 = cv2.imread(filename)
    image3 = cv2.imread(filename)
    names.append(filename)
    names2.append(filename)
    pathlen, filenamed = os.path.split(filename)
    file_count = len(glob.glob1(pathlen,"axial_*.png"))
    image[image <= 90] = 0
    cropped = image[260:430, 210:302]
    n_white_pix = np.append(n_white_pix, np.sum(cropped > 0))
    image2[image2 <= 90] = 0
    cropped2 = image2[210:450, 150:362]
    n_white_pix2 = np.append(n_white_pix2, np.sum(cropped2 > 0))
    image3[image3 >= 10] = 255
    image3[image3 <= 10] = 0
    cropped3 = image3[225:360, 110:402]
    cropped3b = np.invert(cropped3)
    n_white_pix3 = np.append(n_white_pix3, np.sum(cropped3b > 0))
    index += 1
    if index == file_count:
        namesflip = np.flip(names2)
        pixflip = np.flip(n_white_pix3)
        t1 = (np.argmax(pixflip[(int(file_count*0)):(int(file_count*0.3))] > (int(max(pixflip[(int(file_count*0)):(int(file_count*0.3))])* 0.25)))) + int(file_count*0)
        minimum = (np.argmin(n_white_pix[(int(file_count-(t1+10))):(int(file_count-(t1-10)))])) + (int(file_count-(t1+10)))
        midline = cv2.imread(names[minimum])
        path, filenames = os.path.split(names[minimum])
        path2 = os.path.basename(path)
        cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T1_' + filenames, midline)
        t1X = (np.argmax(pixflip[(int(file_count*0)):(int(file_count*0.3))] > (int(max(pixflip[(int(file_count*0)):(int(file_count*0.3))])* 0.20)))) + int(file_count*0)
        minimumX = (np.argmin(n_white_pix[(int(file_count-(t1X+10))):(int(file_count-(t1X-10)))])) + (int(file_count-(t1X+10)))
        midline = cv2.imread(names[minimumX])
        path, filenames = os.path.split(names[minimumX])
        path2 = os.path.basename(path)
        cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T1_' + filenames, midline)
        t1XX = (np.argmax(pixflip[(int(file_count*0)):(int(file_count*0.3))] > (int(max(pixflip[(int(file_count*0)):(int(file_count*0.3))])* 0.10)))) + int(file_count*0)
        minimumXX = (np.argmin(n_white_pix[(int(file_count-(t1XX+5))):(int(file_count-(t1XX-5)))])) + (int(file_count-(t1XX+5)))
        midline = cv2.imread(names[minimumXX])
        path, filenames = os.path.split(names[minimumXX])
        path2 = os.path.basename(path)
        cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T1_' + filenames, midline)
        minimum2 = (np.argmin(n_white_pix[(int(minimum * 0.91)):(int(minimum * 0.94))]) + (int(minimum * 0.92)))
        int2 = (int(names[minimum][71:-4]) - int(names[minimum2][71:-4]))
        if int2 >= (minimum*0.07):
            minimum2 = (np.argmin(n_white_pix[(int(minimum * 0.91)):(int(minimum * 0.94))]) + (int(minimum * 0.92))) + int(int2 - (minimum*0.07))
            midline = cv2.imread(names[minimum2])
            path, filenames = os.path.split(names[minimum2])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T2_' + filenames, midline)
        elif int2 <= (minimum*0.055):
            minimum2 = (np.argmin(n_white_pix[(int(minimum * 0.91)):(int(minimum * 0.94))]) + (int(minimum * 0.92))) - int((minimum*0.055) - int2)
            midline = cv2.imread(names[minimum2])
            path, filenames = os.path.split(names[minimum2])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T2_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum2])
            path, filenames = os.path.split(names[minimum2])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T2_' + filenames, midline)
        minimum2X = (np.argmin(n_white_pix[(int(minimumX * 0.91)):(int(minimumX * 0.94))]) + (int(minimumX * 0.90)))
        int2 = (int(names[minimumX][71:-4]) - int(names[minimum2X][71:-4]))
        if int2 >= (minimumX*0.075):
            minimum2X = (np.argmin(n_white_pix[(int(minimumX * 0.91)):(int(minimumX * 0.94))]) + (int(minimumX * 0.90))) + int(int2 - (minimumX*0.075))
            midline = cv2.imread(names[minimum2X])
            path, filenames = os.path.split(names[minimum2X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T2_' + filenames, midline)
        elif int2 <= (minimumX*0.06):
            minimum2X = (np.argmin(n_white_pix[(int(minimumX * 0.91)):(int(minimumX * 0.94))]) + (int(minimumX * 0.90))) - int((minimumX*0.06) - int2)
            midline = cv2.imread(names[minimum2X])
            path, filenames = os.path.split(names[minimum2X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T2_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum2X])
            path, filenames = os.path.split(names[minimum2X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T2_' + filenames, midline)
        minimum2XX = (np.argmin(n_white_pix[(int(minimum * 0.91)):(int(minimum * 0.94))]) + (int(minimum * 0.91)))
        int2 = (int(names[minimum][71:-4]) - int(names[minimum2XX][71:-4]))
        if int2 >= (minimum*0.075):
            minimum2XX = (np.argmin(n_white_pix[(int(minimum * 0.91)):(int(minimum * 0.94))]) + (int(minimum * 0.91))) + int(int2 - (minimum*0.075))
            midline = cv2.imread(names[minimum2XX])
            path, filenames = os.path.split(names[minimum2XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T2_' + filenames, midline)
        elif int2 <= (minimum*0.06):
            minimum2XX = (np.argmin(n_white_pix[(int(minimum * 0.91)):(int(minimum * 0.94))]) + (int(minimum * 0.91))) - int((minimum*0.06) - int2)
            midline = cv2.imread(names[minimum2XX])
            path, filenames = os.path.split(names[minimum2XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T2_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum2XX])
            path, filenames = os.path.split(names[minimum2XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T2_' + filenames, midline)
        minimum3 = (np.argmin(n_white_pix[(int(minimum * 0.85)):(int(minimum * 0.87))]) + (int(minimum * 0.86)))
        int3 = (int(names[minimum2][71:-4]) - int(names[minimum3][71:-4]))
        if int3 >= (minimum*0.075):
            minimum3 = (np.argmin(n_white_pix[(int(minimum * 0.85)):(int(minimum * 0.87))]) + (int(minimum * 0.86))) + int(int3 - (minimum*0.075))
            midline = cv2.imread(names[minimum3])
            path, filenames = os.path.split(names[minimum3])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T3_' + filenames, midline)
        elif int3 <= (minimum*0.06):
            minimum3 = (np.argmin(n_white_pix[(int(minimum * 0.85)):(int(minimum * 0.87))]) + (int(minimum * 0.86))) - int((minimum*0.06) - int3)
            midline = cv2.imread(names[minimum3])
            path, filenames = os.path.split(names[minimum3])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T3_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum3])
            path, filenames = os.path.split(names[minimum3])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T3_' + filenames, midline)
        minimum3X = (np.argmin(n_white_pix[(int(minimumX * 0.85)):(int(minimumX * 0.87))]) + (int(minimumX * 0.84)))
        int3 = (int(names[minimum2X][71:-4]) - int(names[minimum3X][71:-4]))
        if int3 >= (minimumX*0.075):
            minimum3X = (np.argmin(n_white_pix[(int(minimumX * 0.85)):(int(minimumX * 0.87))]) + (int(minimumX * 0.84))) + int(int3 - (minimumX*0.075))
            midline = cv2.imread(names[minimum3X])
            path, filenames = os.path.split(names[minimum3X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T3_' + filenames, midline)
        elif int3 <= (minimumX*0.06):
            minimum3X = (np.argmin(n_white_pix[(int(minimumX * 0.85)):(int(minimumX * 0.87))]) + (int(minimumX * 0.84))) - int((minimumX*0.06) - int3)
            midline = cv2.imread(names[minimum3X])
            path, filenames = os.path.split(names[minimum3X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T3_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum3X])
            path, filenames = os.path.split(names[minimum3X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T3_' + filenames, midline)
        minimum3XX = (np.argmin(n_white_pix[(int(minimum * 0.85)):(int(minimum * 0.87))]) + (int(minimum * 0.85)))
        int3 = (int(names[minimum2XX][71:-4]) - int(names[minimum3XX][71:-4]))
        if int3 >= (minimum*0.075):
            minimum3XX = (np.argmin(n_white_pix[(int(minimum * 0.85)):(int(minimum * 0.87))]) + (int(minimum * 0.85))) + int(int3 - (minimum*0.075))
            midline = cv2.imread(names[minimum3XX])
            path, filenames = os.path.split(names[minimum3XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T3_' + filenames, midline)
        elif int3 <= (minimum*0.06):
            minimum3XX = (np.argmin(n_white_pix[(int(minimum * 0.85)):(int(minimum * 0.87))]) + (int(minimum * 0.85))) - int((minimum*0.06) - int3)
            midline = cv2.imread(names[minimum3XX])
            path, filenames = os.path.split(names[minimum3XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T3_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum3XX])
            path, filenames = os.path.split(names[minimum3XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T3_' + filenames, midline)
        minimum4 = (np.argmin(n_white_pix[(int(minimum * 0.78)):(int(minimum * 0.80))]) + (int(minimum * 0.79)))
        int4 = (int(names[minimum3][71:-4]) - int(names[minimum4][71:-4]))
        if int4 >= (minimum*0.075):
            minimum4 = (np.argmin(n_white_pix[(int(minimum * 0.78)):(int(minimum * 0.80))]) + (int(minimum * 0.79))) + int(int4 - (minimum*0.075))
            midline = cv2.imread(names[minimum4])
            path, filenames = os.path.split(names[minimum4])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T4_' + filenames, midline)
        elif int4 <= (minimum*0.06):
            minimum4 = (np.argmin(n_white_pix[(int(minimum * 0.78)):(int(minimum * 0.80))]) + (int(minimum * 0.79))) - int((minimum*0.06) - int4)
            midline = cv2.imread(names[minimum4])
            path, filenames = os.path.split(names[minimum4])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T4_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum4])
            path, filenames = os.path.split(names[minimum4])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T4_' + filenames, midline)
        minimum4X = (np.argmin(n_white_pix[(int(minimumX * 0.78)):(int(minimumX * 0.80))]) + (int(minimumX * 0.77)))
        int4 = (int(names[minimum3X][71:-4]) - int(names[minimum4X][71:-4]))
        if int4 >= (minimumX*0.075):
            minimum4X = (np.argmin(n_white_pix[(int(minimumX * 0.78)):(int(minimumX * 0.80))]) + (int(minimumX * 0.77))) + int(int4 - (minimumX*0.075))
            midline = cv2.imread(names[minimum4X])
            path, filenames = os.path.split(names[minimum4X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T4_' + filenames, midline)
        elif int4 <= (minimumX*0.06):
            minimum4X = (np.argmin(n_white_pix[(int(minimumX * 0.78)):(int(minimumX * 0.80))]) + (int(minimumX * 0.77))) - int((minimumX*0.06) - int4)
            midline = cv2.imread(names[minimum4X])
            path, filenames = os.path.split(names[minimum4X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T4_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum4X])
            path, filenames = os.path.split(names[minimum4X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T4_' + filenames, midline)
        minimum4XX = (np.argmin(n_white_pix[(int(minimum * 0.78)):(int(minimum * 0.80))]) + (int(minimum * 0.78)))
        int4 = (int(names[minimum3XX][71:-4]) - int(names[minimum4XX][71:-4]))
        if int4 >= (minimum*0.08):
            minimum4XX = (np.argmin(n_white_pix[(int(minimum * 0.78)):(int(minimum * 0.80))]) + (int(minimum * 0.78))) + int(int4 - (minimum*0.08))
            midline = cv2.imread(names[minimum4XX])
            path, filenames = os.path.split(names[minimum4XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T4_' + filenames, midline)
        elif int4 <= (minimum*0.065):
            minimum4XX = (np.argmin(n_white_pix[(int(minimum * 0.78)):(int(minimum * 0.80))]) + (int(minimum * 0.78))) - int((minimum*0.065) - int4)
            midline = cv2.imread(names[minimum4XX])
            path, filenames = os.path.split(names[minimum4XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T4_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum4XX])
            path, filenames = os.path.split(names[minimum4XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T4_' + filenames, midline)
        minimum5 = (np.argmin(n_white_pix[(int(minimum * 0.71)):(int(minimum * 0.73))]) + (int(minimum * 0.72)))
        int5 = (int(names[minimum4][71:-4]) - int(names[minimum5][71:-4]))
        if int5 >= (minimum*0.075):
            minimum5 = (np.argmin(n_white_pix[(int(minimum * 0.71)):(int(minimum * 0.73))]) + (int(minimum * 0.72))) + int(int5 - (minimum*0.075))
            midline = cv2.imread(names[minimum5])
            path, filenames = os.path.split(names[minimum5])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T5_' + filenames, midline)
        elif int5 <= (minimum*0.06):
            minimum5 = (np.argmin(n_white_pix[(int(minimum * 0.71)):(int(minimum * 0.73))]) + (int(minimum * 0.72))) - int((minimum*0.06) - int5)
            midline = cv2.imread(names[minimum5])
            path, filenames = os.path.split(names[minimum5])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T5_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum5])
            path, filenames = os.path.split(names[minimum5])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T5_' + filenames, midline)
        minimum5X = (np.argmin(n_white_pix[(int(minimumX * 0.71)):(int(minimumX * 0.73))]) + (int(minimumX * 0.70)))
        int5 = (int(names[minimum4X][71:-4]) - int(names[minimum5X][71:-4]))
        if int5 >= (minimumX*0.08):
            minimum5X = (np.argmin(n_white_pix[(int(minimumX * 0.71)):(int(minimumX * 0.73))]) + (int(minimumX * 0.70))) + int(int5 - (minimumX*0.08))
            midline = cv2.imread(names[minimum5X])
            path, filenames = os.path.split(names[minimum5X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T5_' + filenames, midline)
        elif int5 <= (minimumX*0.065):
            minimum5X = (np.argmin(n_white_pix[(int(minimumX * 0.71)):(int(minimumX * 0.73))]) + (int(minimumX * 0.70))) - int((minimumX*0.065) - int5)
            midline = cv2.imread(names[minimum5X])
            path, filenames = os.path.split(names[minimum5X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T5_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum5X])
            path, filenames = os.path.split(names[minimum5X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T5_' + filenames, midline)
        minimum5XX = (np.argmin(n_white_pix[(int(minimum * 0.71)):(int(minimum * 0.73))]) + (int(minimum * 0.71)))
        int5 = (int(names[minimum4XX][71:-4]) - int(names[minimum5XX][71:-4]))
        if int5 >= (minimum*0.085):
            minimum5XX = (np.argmin(n_white_pix[(int(minimum * 0.71)):(int(minimum * 0.73))]) + (int(minimum * 0.71))) + int(int5 - (minimum*0.085))
            midline = cv2.imread(names[minimum5XX])
            path, filenames = os.path.split(names[minimum5XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T5_' + filenames, midline)
        elif int5 <= (minimum*0.07):
            minimum5XX = (np.argmin(n_white_pix[(int(minimum * 0.71)):(int(minimum * 0.73))]) + (int(minimum * 0.71))) - int((minimum*0.07) - int5)
            midline = cv2.imread(names[minimum5XX])
            path, filenames = os.path.split(names[minimum5XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T5_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum5XX])
            path, filenames = os.path.split(names[minimum5XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T5_' + filenames, midline)
        minimum6 = (np.argmin(n_white_pix2[(int(minimum * 0.62)):(int(minimum * 0.64))]) + (int(minimum * 0.63)))
        int6 = (int(names[minimum5][71:-4]) - int(names[minimum6][71:-4]))
        if int6 >= (minimum*0.08):
            minimum6 = (np.argmin(n_white_pix2[(int(minimum * 0.62)):(int(minimum * 0.64))]) + (int(minimum * 0.63))) + int(int6 - (minimum*0.08))
            midline = cv2.imread(names[minimum6])
            path, filenames = os.path.split(names[minimum6])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T6_' + filenames, midline)
        elif int6 <= (minimum*0.065):
            minimum6 = (np.argmin(n_white_pix2[(int(minimum * 0.62)):(int(minimum * 0.64))]) + (int(minimum * 0.63))) - int((minimum*0.065) - int6)
            midline = cv2.imread(names[minimum6])
            path, filenames = os.path.split(names[minimum6])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T6_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum6])
            path, filenames = os.path.split(names[minimum6])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T6_' + filenames, midline)
        minimum6X = (np.argmin(n_white_pix2[(int(minimumX * 0.62)):(int(minimumX * 0.64))]) + (int(minimumX * 0.61)))
        int6 = (int(names[minimum5X][71:-4]) - int(names[minimum6X][71:-4]))
        if int6 >= (minimumX*0.08):
            minimum6X = (np.argmin(n_white_pix2[(int(minimumX * 0.62)):(int(minimumX * 0.64))]) + (int(minimumX * 0.61))) + int(int6 - (minimumX*0.08))
            midline = cv2.imread(names[minimum6X])
            path, filenames = os.path.split(names[minimum6X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T6_' + filenames, midline)
        elif int6 <= (minimumX*0.065):
            minimum6X = (np.argmin(n_white_pix2[(int(minimumX * 0.62)):(int(minimumX * 0.64))]) + (int(minimumX * 0.61))) - int((minimumX*0.065) - int6)
            midline = cv2.imread(names[minimum6X])
            path, filenames = os.path.split(names[minimum6X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T6_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum6X])
            path, filenames = os.path.split(names[minimum6X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T6_' + filenames, midline)
        minimum6XX = (np.argmin(n_white_pix2[(int(minimum * 0.62)):(int(minimum * 0.64))]) + (int(minimum * 0.61)))
        int6 = (int(names[minimum5X][71:-4]) - int(names[minimum6X][71:-4]))
        if int6 >= (minimum*0.09):
            minimum6XX = (np.argmin(n_white_pix2[(int(minimum * 0.62)):(int(minimum * 0.64))]) + (int(minimum * 0.61))) + int(int6 - (minimum*0.09))
            midline = cv2.imread(names[minimum6XX])
            path, filenames = os.path.split(names[minimum6XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T6_' + filenames, midline)
        elif int6 <= (minimum*0.075):
            minimum6XX = (np.argmin(n_white_pix2[(int(minimum * 0.62)):(int(minimum * 0.64))]) + (int(minimum * 0.61))) - int((minimum*0.075) - int6)
            midline = cv2.imread(names[minimum6XX])
            path, filenames = os.path.split(names[minimum6XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T6_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum6XX])
            path, filenames = os.path.split(names[minimum6XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T6_' + filenames, midline)
        minimum7 = (np.argmin(n_white_pix2[(int(minimum * 0.55)):(int(minimum * 0.57))]) + (int(minimum * 0.56)))
        int7 = (int(names[minimum6][71:-4]) - int(names[minimum7][71:-4]))
        if int7 >= (minimum*0.08):
            minimum7 = (np.argmin(n_white_pix2[(int(minimum * 0.55)):(int(minimum * 0.57))]) + (int(minimum * 0.56))) + int(int7 - (minimum*0.08))
            midline = cv2.imread(names[minimum7])
            path, filenames = os.path.split(names[minimum7])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T7_' + filenames, midline)
        elif int7 <= (minimum*0.065):
            minimum7 = (np.argmin(n_white_pix2[(int(minimum * 0.55)):(int(minimum * 0.57))]) + (int(minimum * 0.56))) - int((minimum*0.065) - int7)
            midline = cv2.imread(names[minimum7])
            path, filenames = os.path.split(names[minimum7])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T7_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum7])
            path, filenames = os.path.split(names[minimum7])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T7_' + filenames, midline)
        minimum7X = (np.argmin(n_white_pix2[(int(minimumX * 0.55)):(int(minimumX * 0.57))]) + (int(minimumX * 0.54)))
        int7 = (int(names[minimum6X][71:-4]) - int(names[minimum7X][71:-4]))
        if int7 >= (minimumX*0.085):
            minimum7X = (np.argmin(n_white_pix2[(int(minimumX * 0.55)):(int(minimumX * 0.57))]) + (int(minimumX * 0.54))) + int(int7 - (minimumX*0.085))
            midline = cv2.imread(names[minimum7X])
            path, filenames = os.path.split(names[minimum7X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T7_' + filenames, midline)
        elif int7 <= (minimumX*0.07):
            minimum7X = (np.argmin(n_white_pix2[(int(minimumX * 0.55)):(int(minimumX * 0.57))]) + (int(minimumX * 0.54))) - int((minimumX*0.07) - int7)
            midline = cv2.imread(names[minimum7X])
            path, filenames = os.path.split(names[minimum7X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T7_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum7X])
            path, filenames = os.path.split(names[minimum7X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T7_' + filenames, midline)
        minimum7XX = (np.argmin(n_white_pix2[(int(minimum * 0.55)):(int(minimum * 0.57))]) + (int(minimum * 0.54)))
        int7 = (int(names[minimum6XX][71:-4]) - int(names[minimum7XX][71:-4]))
        if int7 >= (minimum*0.095):
            minimum7XX = (np.argmin(n_white_pix2[(int(minimum * 0.55)):(int(minimum * 0.57))]) + (int(minimum * 0.54))) + int(int7 - (minimum*0.095))
            midline = cv2.imread(names[minimum7XX])
            path, filenames = os.path.split(names[minimum7XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T7_' + filenames, midline)
        elif int7 <= (minimum*0.08):
            minimum7XX = (np.argmin(n_white_pix2[(int(minimum * 0.55)):(int(minimum * 0.57))]) + (int(minimum * 0.54))) - int((minimum*0.08) - int7)
            midline = cv2.imread(names[minimum7XX])
            path, filenames = os.path.split(names[minimum7XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T7_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum7XX])
            path, filenames = os.path.split(names[minimum7XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T7_' + filenames, midline)
        minimum8 = (np.argmin(n_white_pix2[(int(minimum * 0.45)):(int(minimum * 0.49))]) + (int(minimum * 0.46)))
        int8 = (int(names[minimum7][71:-4]) - int(names[minimum8][71:-4]))
        if int8 >= (minimum*0.085):
            minimum8 = (np.argmin(n_white_pix2[(int(minimum * 0.45)):(int(minimum * 0.49))]) + (int(minimum * 0.46))) + int(int8 - (minimum*0.085))
            midline = cv2.imread(names[minimum8])
            path, filenames = os.path.split(names[minimum8])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T8_' + filenames, midline)
        elif int8 <= (minimum*0.07):
            minimum8 = (np.argmin(n_white_pix2[(int(minimum * 0.45)):(int(minimum * 0.49))]) + (int(minimum * 0.46))) - int((minimum*0.07) - int8)
            midline = cv2.imread(names[minimum8])
            path, filenames = os.path.split(names[minimum8])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T8_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum8])
            path, filenames = os.path.split(names[minimum8])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T8_' + filenames, midline)
        minimum8X = (np.argmin(n_white_pix2[(int(minimumX * 0.45)):(int(minimumX * 0.49))]) + (int(minimumX * 0.44)))
        int8 = (int(names[minimum7X][71:-4]) - int(names[minimum8X][71:-4]))
        if int8 >= (minimumX*0.095):
            minimum8X = (np.argmin(n_white_pix2[(int(minimumX * 0.45)):(int(minimumX * 0.49))]) + (int(minimumX * 0.44))) + int(int8 - (minimumX*0.095))
            midline = cv2.imread(names[minimum8X])
            path, filenames = os.path.split(names[minimum8X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T8_' + filenames, midline)
        elif int8 <= (minimumX*0.08):
            minimum8X = (np.argmin(n_white_pix2[(int(minimumX * 0.45)):(int(minimumX * 0.49))]) + (int(minimumX * 0.44))) - int((minimumX*0.08) - int8)
            midline = cv2.imread(names[minimum8X])
            path, filenames = os.path.split(names[minimum8X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T8_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum8X])
            path, filenames = os.path.split(names[minimum8X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T8_' + filenames, midline)
        minimum8XX = (np.argmin(n_white_pix2[(int(minimum * 0.45)):(int(minimum * 0.49))]) + (int(minimum * 0.44)))
        int8 = (int(names[minimum7XX][71:-4]) - int(names[minimum8XX][71:-4]))
        if int8 >= (minimumX*0.1):
            minimum8XX = (np.argmin(n_white_pix2[(int(minimum * 0.45)):(int(minimum * 0.49))]) + (int(minimum * 0.44))) + int(int8 - (minimum*0.1))
            midline = cv2.imread(names[minimum8XX])
            path, filenames = os.path.split(names[minimum8XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T8_' + filenames, midline)
        elif int8 <= (minimum*0.085):
            minimum8XX = (np.argmin(n_white_pix2[(int(minimum * 0.45)):(int(minimum * 0.49))]) + (int(minimum * 0.44))) - int((minimum*0.085) - int8)
            midline = cv2.imread(names[minimum8XX])
            path, filenames = os.path.split(names[minimum8XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T8_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum8XX])
            path, filenames = os.path.split(names[minimum8XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T8_' + filenames, midline)
        minimum9 = (np.argmin(n_white_pix2[(int(minimum * 0.37)):(int(minimum * 0.41))]) + (int(minimum * 0.39)))
        int9 = (int(names[minimum8][71:-4]) - int(names[minimum9][71:-4]))
        if int9 >= (minimum*0.09):
            minimum9 = (np.argmin(n_white_pix2[(int(minimum * 0.37)):(int(minimum * 0.41))]) + (int(minimum * 0.39))) + int(int9 - (minimum*0.09))
            midline = cv2.imread(names[minimum9])
            path, filenames = os.path.split(names[minimum9])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T9_' + filenames, midline)
        elif int9 <= (minimum*0.075):
            minimum9 = (np.argmin(n_white_pix2[(int(minimum * 0.37)):(int(minimum * 0.41))]) + (int(minimum * 0.39))) - int((minimum*0.075) - int9)
            midline = cv2.imread(names[minimum9])
            path, filenames = os.path.split(names[minimum9])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T9_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum9])
            path, filenames = os.path.split(names[minimum9])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T9_' + filenames, midline)
        minimum9X = (np.argmin(n_white_pix2[(int(minimumX * 0.37)):(int(minimumX * 0.41))]) + (int(minimumX * 0.35)))
        int9 = (int(names[minimum8X][71:-4]) - int(names[minimum9X][71:-4]))
        if int9 >= (minimumX*0.105):
            minimum9X = (np.argmin(n_white_pix2[(int(minimumX * 0.37)):(int(minimumX * 0.41))]) + (int(minimumX * 0.35))) + int(int9 - (minimumX*0.105))
            midline = cv2.imread(names[minimum9X])
            path, filenames = os.path.split(names[minimum9X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T9_' + filenames, midline)
        elif int9 <= (minimumX*0.085):
            minimum9X = (np.argmin(n_white_pix2[(int(minimumX * 0.37)):(int(minimumX * 0.41))]) + (int(minimumX * 0.35))) - int((minimumX*0.085) - int9)
            midline = cv2.imread(names[minimum9X])
            path, filenames = os.path.split(names[minimum9X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T9_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum9X])
            path, filenames = os.path.split(names[minimum9X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T9_' + filenames, midline)
        minimum9XX = (np.argmin(n_white_pix2[(int(minimum * 0.37)):(int(minimum * 0.41))]) + (int(minimum * 0.37)))
        int9 = (int(names[minimum8X][71:-4]) - int(names[minimum9X][71:-4]))
        if int9 >= (minimum*0.105):
            minimum9XX = (np.argmin(n_white_pix2[(int(minimum * 0.37)):(int(minimum * 0.41))]) + (int(minimum * 0.37))) + int(int9 - (minimum*0.105))
            midline = cv2.imread(names[minimum9XX])
            path, filenames = os.path.split(names[minimum9XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T9_' + filenames, midline)
        elif int9 <= (minimum*0.09):
            minimum9XX = (np.argmin(n_white_pix2[(int(minimum * 0.37)):(int(minimum * 0.41))]) + (int(minimum * 0.37))) - int((minimum*0.09) - int9)
            midline = cv2.imread(names[minimum9XX])
            path, filenames = os.path.split(names[minimum9XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T9_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum9XX])
            path, filenames = os.path.split(names[minimum9XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T9_' + filenames, midline)
        minimum10 = (np.argmin(n_white_pix2[(int(minimum * 0.29)):(int(minimum * 0.33))]) + (int(minimum * 0.31)))
        int10 = (int(names[minimum9][71:-4]) - int(names[minimum10][71:-4]))
        if int10 >= (minimum*0.095):
            minimum10 = (np.argmin(n_white_pix2[(int(minimum * 0.29)):(int(minimum * 0.33))]) + (int(minimum * 0.31))) + int(int10 - (minimum*0.095))
            midline = cv2.imread(names[minimum10])
            path, filenames = os.path.split(names[minimum10])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T10_' + filenames, midline)
        elif int10 <= (minimum*0.08):
            minimum10 = (np.argmin(n_white_pix2[(int(minimum * 0.29)):(int(minimum * 0.33))]) + (int(minimum * 0.31))) - int((minimum*0.08) - int10)
            midline = cv2.imread(names[minimum10])
            path, filenames = os.path.split(names[minimum10])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T10_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum10])
            path, filenames = os.path.split(names[minimum10])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T10_' + filenames, midline)
        minimum10X = (np.argmin(n_white_pix2[(int(minimumX * 0.29)):(int(minimumX * 0.33))]) + (int(minimumX * 0.27)))
        int10 = (int(names[minimum9X][71:-4]) - int(names[minimum10X][71:-4]))
        if int10 >= (minimumX*0.11):
            minimum10X = (np.argmin(n_white_pix2[(int(minimumX * 0.29)):(int(minimumX * 0.33))]) + (int(minimumX * 0.27))) + int(int10 - (minimumX*0.11))
            midline = cv2.imread(names[minimum10X])
            path, filenames = os.path.split(names[minimum10X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T10_' + filenames, midline)
        elif int10 <= (minimumX*0.09):
            minimum10X = (np.argmin(n_white_pix2[(int(minimumX * 0.29)):(int(minimumX * 0.33))]) + (int(minimumX * 0.27))) - int((minimumX*0.09) - int10)
            midline = cv2.imread(names[minimum10X])
            path, filenames = os.path.split(names[minimum10X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T10_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum10X])
            path, filenames = os.path.split(names[minimum10X])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T10_' + filenames, midline)
        minimum10XX = (np.argmin(n_white_pix2[(int(minimum * 0.29)):(int(minimum * 0.33))]) + (int(minimum * 0.29)))
        int10 = (int(names[minimum9XX][71:-4]) - int(names[minimum10XX][71:-4]))
        if int10 >= (minimum*0.11):
            minimum10XX = (np.argmin(n_white_pix2[(int(minimum * 0.29)):(int(minimum * 0.33))]) + (int(minimum * 0.29))) + int(int10 - (minimum*0.11))
            midline = cv2.imread(names[minimum10XX])
            path, filenames = os.path.split(names[minimum10XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T10_' + filenames, midline)
        elif int10 <= (minimum*0.095):
            minimum10XX = (np.argmin(n_white_pix2[(int(minimum * 0.29)):(int(minimum * 0.33))]) + (int(minimum * 0.29))) - int((minimum*0.095) - int10)
            midline = cv2.imread(names[minimum10XX])
            path, filenames = os.path.split(names[minimum10XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T10_' + filenames, midline)
        else:
            midline = cv2.imread(names[minimum10XX])
            path, filenames = os.path.split(names[minimum10XX])
            path2 = os.path.basename(path)        
            cv2.imwrite('/home/john/Documents/MESArthritis/Ax_Cor_Sag_PNG/' + path2 + '/' + 'T10_' + filenames, midline)

        index = 0
        n_white_pix = []
        n_white_pix2 = []
        n_white_pix3 = []
        names = []
        names2 = []
        file_count = []
    else:
        continue

In [ ]:
### Load Essential Packages ###
import numpy as np
import pandas as pd
import os
import cv2
from imutils import paths
import argparse
import glob
from glob import iglob

n_white_pixUpper = []
n_white_pixLower = []
names = []
index = 0

###Sagittal###
for filename in sorted(glob.glob('/media/john/Hollander4TB/vBMD/3D_Volume/**/sagittal*.png'), key=lambda name: (name[40:51], int(os.path.basename(name)[17:-4]))):
    image = cv2.imread(filename)
    names.append(filename)
    pathlen, filenamed = os.path.split(filename)
    file_count = len(glob.glob1(pathlen,"sagittal_*.png"))
    image[image <= 130] = 0
    croppedUpper = image[50:370, 200:512]
    n_white_pixUpper = np.append(n_white_pixUpper, np.sum(croppedUpper > 0))
    croppedLower = image[320:640, 200:512]
    n_white_pixLower = np.append(n_white_pixLower, np.sum(croppedLower > 0))
    index += 1
    if index == file_count:
        namesflipUpper = np.flip(names)
        pixflipUpper = np.flip(n_white_pixUpper)
        namesflipLower = np.flip(names)
        pixflipLower = np.flip(n_white_pixLower)
        leftUpper = (np.argmax(n_white_pixUpper[(int(file_count*0.3)):(int(file_count*0.7))] > (int(max(n_white_pixUpper[(int(file_count*0.3)):(int(file_count*0.7))])* 0.75)))) + int(file_count*0.3)
        rightUpper = 512 - ((np.argmax(pixflipUpper[(int(file_count*0.3)):(int(file_count*0.7))] > (int(max(pixflipUpper[(int(file_count*0.3)):(int(file_count*0.7))])* 0.45)))) + int(file_count*0.3))
        leftLower = (np.argmax(n_white_pixLower[(int(file_count*0.3)):(int(file_count*0.7))] > (int(max(n_white_pixLower[(int(file_count*0.3)):(int(file_count*0.7))])* 0.5)))) + int(file_count*0.3)
        rightLower = 512 - ((np.argmax(pixflipLower[(int(file_count*0.3)):(int(file_count*0.7))] > (int(max(pixflipLower[(int(file_count*0.3)):(int(file_count*0.7))])* 0.5)))) + int(file_count*0.3))
        middleUpper = int((rightUpper+leftUpper)/2)
        middleLower = int((rightLower+leftLower)/2)
        midlineUpper = cv2.imread(names[middleUpper])
        midlineLower = cv2.imread(names[middleLower])
        pathUpper, filenamesUpper = os.path.split(names[middleUpper])
        pathLower, filenamesLower = os.path.split(names[middleLower])
        path2Upper = os.path.basename(pathUpper)
        path2Lower = os.path.basename(pathLower)
        cv2.imwrite('/media/john/Hollander4TB/vBMD/Sagittal/' + path2Upper + '/' + 'SagUpper_' + filenamesUpper, midlineUpper)
        cv2.imwrite('/media/john/Hollander4TB/vBMD/Sagittal/' + path2Lower + '/' + 'SagLower_' + filenamesLower, midlineLower)

        index = 0
        n_white_pixUpper = []
        n_white_pixLower = []
        names = []
        file_count = []
    else:
        continue

In [ ]:
### Load Essential Packages ###
import glob
import json
import os
import os.path as osp

import numpy as np
import PIL.Image
import cv2
import labelme

### Export Binary Mask Layer for PM, SAT, and Both ###
DATA_DIR = '/home/john/Documents/MESArthritis/Development_Bone/train'
OUT_DIR = '/home/john/Documents/MESArthritis/Ground_Truth_Bone/train'
class_names = []
class_name_to_id = {}
for i, line in enumerate(open(DATA_DIR + '/labels.txt').readlines()):
    class_id = i
    class_name = line.strip()
    class_name_to_id[class_name] = class_id
#    if class_id == -1:
#        class_name == '__ignore__'
#        continue
#    elif class_id == 0:
#        class_name == '_background_'
    class_names.append(class_name)
class_names = tuple(class_names)
print('class_names:', class_names)
out_class_names_file = osp.join(DATA_DIR, 'class_names.txt')
with open(out_class_names_file, 'w') as f:
    f.writelines('\n'.join(class_names))
print('Saved class_names:', out_class_names_file)

if osp.exists(OUT_DIR):
    print('Output directory already exists:', OUT_DIR)
    quit(1)
os.makedirs(OUT_DIR, exist_ok=True)
for label_file in sorted(glob.glob(osp.join(DATA_DIR, '*.json'))):
    with open(label_file) as f:
        base = osp.splitext(osp.basename(label_file))[0]
        data = json.load(f)
        img_file = osp.join(osp.dirname(label_file), data['imagePath'])
        img = np.asarray(PIL.Image.open(img_file))
        lbl, _ = labelme.utils.shapes_to_label(
            img_shape=img.shape,
            shapes=data['shapes'],
            label_name_to_value=class_name_to_id,
        )
        instance1 = np.copy(lbl)
        pos_1 = np.where(lbl==0)
        instance1[pos_1] = 0

        instance1 = instance1*255
        os.makedirs(osp.join(OUT_DIR, base[:11]), exist_ok=True)
        os.makedirs(osp.join(OUT_DIR, base[:11],'images'), exist_ok=True)
        PIL.Image.fromarray(img).save(osp.join(OUT_DIR, base[:11], 'images', base + '.png'))
        os.makedirs(osp.join(OUT_DIR, base[:11], 'masks'), exist_ok=True)
        cv2.imwrite(osp.join(OUT_DIR, base[:11], 'masks', base + '_ROI' + '.png'), instance1)

In [ ]:
# import package
import labelme2coco

# set directory that contains labelme annotations and image files
labelme_folder = "/home/john/Documents/MESArthritis/Development_Bone_SagUpper/test"

# set export dir
export_dir = "/home/john/Documents/MESArthritis/Coco_Anno/"

# convert labelme annotations to coco
labelme2coco.convert(labelme_folder, export_dir)

In [ ]:
### Load Essential Packages ###
import pixellib
from pixellib.custom_train import instance_custom_training

### Visualize a Radom Sample from the Training Group ###
vis_img = instance_custom_training()
vis_img.load_dataset("/home/john/Documents/MESArthritis/Development_Bone")
vis_img.visualize_sample()

In [ ]:
###AXIAL###

from PIL import Image
import numpy as np
import cv2
import os
import glob
from glob import iglob

for filename in sorted(glob.glob('/home/john/Documents/MESArthritis/Inference_Bone/Infer_Axial_Cropped_ALL/*.png')):
    original = np.asarray(Image.open(filename))
    cropped = original[240:450, 190:322]

    right = 190
    left = 190
    top = 240
    bottom = 62

    cropped = Image.fromarray(np.uint8(cropped))
    width, height = cropped.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(cropped.mode, (new_width, new_height), (0, 0, 0))
    result.paste(cropped, (left, top))

    result.save(filename)

In [ ]:
###SAGITTAL###

from PIL import Image
import numpy as np
import cv2
import os
import glob
from glob import iglob

for filename in sorted(glob.glob('/home/john/Documents/MESArthritis/Development_Bone_SagLower_Cropped/test/*.png')):
    original = np.asarray(Image.open(filename))
    cropped = original[200:525, 250:450]

    right = 62
    left = 250
    top = 200
    bottom = 115

    cropped = Image.fromarray(np.uint8(cropped))
    width, height = cropped.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(cropped.mode, (new_width, new_height), (0))
    result.paste(cropped, (left, top))

    result.save(filename)

In [ ]:
import imgaug as ia
import imgaug.augmenters as iaa
# Define our augmentation pipeline.
augmentation_sequence_pipeline = iaa.Sequential([
    iaa.CropToFixedSize(width=125, height=350, position="center"),
    iaa.PadToFixedSize(width=512, height=512, position="center")
    ])

In [ ]:
###Axial###

In [ ]:
### Load Essential Packages ###
import pixellib
from pixellib.custom_train import instance_custom_training

### Train the MASK R-CNN Model from the Groundtruth Images ###
train_maskrcnn = instance_custom_training()
train_maskrcnn.modelConfig(network_backbone = "resnet50", num_classes = 1, batch_size = 12)
train_maskrcnn.load_pretrained_model("/home/john/Documents/MESArthritis/mask_rcnn_coco.h5")
train_maskrcnn.load_dataset("/home/john/Documents/MESArthritis/Development_Bone_Cropped_2")
train_maskrcnn.train_model(num_epochs = 10, augmentation=False,  path_trained_models = "/home/john/Documents/MESArthritis/Models/Spine_Axial_Whole/")

In [ ]:
### Load Essential Packages ###
import pixellib
from pixellib.custom_train import instance_custom_training

### Evaluate performance of the MASK R-CNN Model using the Test/Holdout Group ###
train_maskrcnn = instance_custom_training()
train_maskrcnn.modelConfig(network_backbone = "resnet50", num_classes= 1, batch_size = 1)
train_maskrcnn.load_pretrained_model("/home/john/Documents/MESArthritis/mask_rcnn_coco.h5")
train_maskrcnn.load_dataset("/home/john/Documents/MESArthritis/Development_Bone_Cropped_2")
train_maskrcnn.evaluate_model("/home/john/Documents/MESArthritis/Models/Spine_Axial_Whole/Spine_Cropped_2_Batch12_E6.h5")

In [ ]:
### Load Essential Packages ###
from pixellib.instance import custom_segmentation

### Custom Segmentation of PM right (RMA), PM left (LMA), and SFA ###
segment_image = custom_segmentation()
segment_image.inferConfig(num_classes = 1, class_names= ["BG", "AXIAL"], detection_threshold = 0.9, network_backbone = "resnet50")
segment_image.load_model("/home/john/Documents/MESArthritis/Models/Spine_Axial_Whole/Spine_Cropped_2_Batch12_E6.h5")

#segment_image.segmentImage("/home/john/Documents/MESArthritis/Other/MESA6018599_T9_axial_IM-3431_222.png", show_bboxes=True, output_image_name="/home/john/Documents/MESArthritis/Other/MESA6018599_T9_axial_IM-3431_222_2.png")

In [ ]:
###Sagittal Lower###

In [ ]:
### Load Essential Packages ###
import pixellib
from pixellib.custom_train import instance_custom_training

### Train the MASK R-CNN Model from the Groundtruth Images ###
train_maskrcnn = instance_custom_training()
train_maskrcnn.modelConfig(network_backbone = "resnet50", num_classes = 1, batch_size = 8)
train_maskrcnn.load_pretrained_model("/home/john/Documents/MESArthritis/mask_rcnn_coco.h5")
train_maskrcnn.load_dataset("/home/john/Documents/MESArthritis/Development_Bone_SagLower_C1")
train_maskrcnn.train_model(num_epochs = 20, augmentation=False,  path_trained_models = "/home/john/Documents/MESArthritis/Models/Spine_Sagittal/SagLower_C1/")

In [ ]:
### Load Essential Packages ###
import pixellib
from pixellib.custom_train import instance_custom_training

### Evaluate performance of the MASK R-CNN Model using the Test/Holdout Group ###
train_maskrcnn = instance_custom_training()
train_maskrcnn.modelConfig(network_backbone = "resnet50", num_classes= 1, batch_size = 1)
train_maskrcnn.load_pretrained_model("/home/john/Documents/MESArthritis/mask_rcnn_coco.h5")
train_maskrcnn.load_dataset("/home/john/Documents/MESArthritis/Development_Bone_SagLower_C1")
train_maskrcnn.evaluate_model("/home/john/Documents/MESArthritis/Models/Spine_Sagittal/SagLower_C1/mask_rcnn_model.012-0.351750.h5")

In [ ]:
### Load Essential Packages ###
from pixellib.instance import custom_segmentation

### Custom Segmentation of PM right (RMA), PM left (LMA), and SFA ###
segment_image = custom_segmentation()
segment_image.inferConfig(num_classes = 1, class_names= ["BG", "AXIAL"], detection_threshold = 0.9, network_backbone = "resnet50")
segment_image.load_model("/home/john/Documents/MESArthritis/Models/Spine_Sagittal/SagLower_C1/mask_rcnn_model.012-0.351750.h5")

#segment_image.segmentImage("/home/john/Documents/MESArthritis/Other/sagittal_IM-0162_269.png", show_bboxes=True, output_image_name="/home/john/Documents/MESArthritis/Other/sagittal_IM-0162_269_TEST_LOWER.png")

In [ ]:
###Sagittal Upper###

In [ ]:
### Load Essential Packages ###
import pixellib
from pixellib.custom_train import instance_custom_training

### Train the MASK R-CNN Model from the Groundtruth Images ###
train_maskrcnn = instance_custom_training()
train_maskrcnn.modelConfig(network_backbone = "resnet50", num_classes = 1, batch_size = 8)
train_maskrcnn.load_pretrained_model("/home/john/Documents/MESArthritis/mask_rcnn_coco.h5")
train_maskrcnn.load_dataset("/home/john/Documents/MESArthritis/Development_Bone_SagUpper_C1")
train_maskrcnn.train_model(num_epochs = 20, augmentation=False,  path_trained_models = "/home/john/Documents/MESArthritis/Models/Spine_Sagittal/SagUpper_C1/")

In [ ]:
### Load Essential Packages ###
import pixellib
from pixellib.custom_train import instance_custom_training

### Evaluate performance of the MASK R-CNN Model using the Test/Holdout Group ###
train_maskrcnn = instance_custom_training()
train_maskrcnn.modelConfig(network_backbone = "resnet50", num_classes= 1, batch_size = 1)
train_maskrcnn.load_pretrained_model("/home/john/Documents/MESArthritis/mask_rcnn_coco.h5")
train_maskrcnn.load_dataset("/home/john/Documents/MESArthritis/Development_Bone_SagUpper_C1")
train_maskrcnn.evaluate_model("/home/john/Documents/MESArthritis/Models/Spine_Sagittal/SagUpper_C1/mask_rcnn_model.017-0.512719.h5")

In [ ]:
### Load Essential Packages ###
from pixellib.instance import custom_segmentation

### Custom Segmentation of PM right (RMA), PM left (LMA), and SFA ###
segment_image = custom_segmentation()
segment_image.inferConfig(num_classes = 1, class_names= ["BG", "AXIAL"], detection_threshold = 0.9, network_backbone = "resnet50")
segment_image.load_model("/home/john/Documents/MESArthritis/Models/Spine_Sagittal/SagUpper_C1/mask_rcnn_model.017-0.512719.h5")

#segment_image.segmentImage("/home/john/Documents/MESArthritis/Other/sagittal_IM-0162_269.png", show_bboxes=True, output_image_name="/home/john/Documents/MESArthritis/Other/sagittal_IM-0162_269_TEST_UPPER.png")

In [ ]:
###Axial###

In [ ]:
### Load Essential Packages ###
import numpy as np
from PIL import Image
import cv2
import os

inputdir = '/home/john/Documents/MESArthritis/Inference_Bone/Infer_Axial_Cropped_ALL/'
outdir = '/home/john/Documents/MESArthritis/Inference_Bone_2/Output_Axial_ALL/'
outdir1 = '/home/john/Documents/MESArthritis/Inference_Bone_2/Mask_Infer_Axial_ALL/Infer_Mask_'
outdir2 = '/home/john/Documents/MESArthritis/Inference_Bone_2/Mask_Infer_Axial_ALL/Inference_'

test_list = [ f for f in  os.listdir(inputdir)]

### Inference of PM right (PMR), PM left (PML), and SAT Images Based on Mask R-CNN Model ###
try:
    for f in sorted (test_list):
        if f.endswith('.png'):
            segmask, output = segment_image.segmentImage(inputdir + f, extract_segmented_objects = False, mask_points_values = True, show_bboxes=False)
            class_list = segmask['class_ids'].tolist()
            class_str = ''.join(str(x) for x in class_list)
            a = [ [individualArray] for individualArray in segmask['masks'] ]

            cv2.imwrite(outdir + f, output)
            a1 = np.array(a)
            try:
                for i in a1.tolist():
                    mask1 = np.asarray(Image.open(inputdir + f))
                    mask11 =  mask1[:,:,:3]
                    img1 = cv2.fillPoly(mask1, np.array(i), color=(0, 0, 0))
                    maskB = Image.fromarray(mask11)
                for i in a1.tolist():
                    mask2 = np.asarray(Image.open(inputdir + f))
                    mask22 =  mask2[:,:,:3]
                    img2 = cv2.fillPoly(mask2, np.array(i), color=(255, 255, 255))
                    maskW = Image.fromarray(mask22)
                    Infer_Mask = np.subtract(maskW, maskB)
                    Inference = np.subtract(np.asarray(Image.open(inputdir + f))[:,:,:3], maskB)
                    cv2.imwrite(outdir1 + f, Infer_Mask)
                    cv2.imwrite(outdir2 + f, Inference)
            except Exception:
                pass
except TypeError:
    pass

In [ ]:
###Sagittal Lower###

In [ ]:
### Load Essential Packages ###
import numpy as np
from PIL import Image
import cv2
import os
import itertools
import pandas as pd
import operator

inputdir = '/home/john/Documents/MESArthritis/Inference_Bone_SagLower/Infer_Sag_ALL/'
outdir = '/home/john/Documents/MESArthritis/Inference_Bone_SagLower_3/Output_Sag_ALL/'
outdir1 = '/home/john/Documents/MESArthritis/Inference_Bone_SagLower_3/Mask_Infer_Sag_ALL/Infer_Mask_T6_'
outdir2 = '/home/john/Documents/MESArthritis/Inference_Bone_SagLower_3/Mask_Infer_Sag_ALL/Inference_T6_'
outdir3 = '/home/john/Documents/MESArthritis/Inference_Bone_SagLower_3/Mask_Infer_Sag_ALL/Infer_Mask_T7_'
outdir4 = '/home/john/Documents/MESArthritis/Inference_Bone_SagLower_3/Mask_Infer_Sag_ALL/Inference_T7_'
outdir5 = '/home/john/Documents/MESArthritis/Inference_Bone_SagLower_3/Mask_Infer_Sag_ALL/Infer_Mask_T8_'
outdir6 = '/home/john/Documents/MESArthritis/Inference_Bone_SagLower_3/Mask_Infer_Sag_ALL/Inference_T8_'
outdir7 = '/home/john/Documents/MESArthritis/Inference_Bone_SagLower_3/Mask_Infer_Sag_ALL/Infer_Mask_T9_'
outdir8 = '/home/john/Documents/MESArthritis/Inference_Bone_SagLower_3/Mask_Infer_Sag_ALL/Inference_T9_'
outdir9 = '/home/john/Documents/MESArthritis/Inference_Bone_SagLower_3/Mask_Infer_Sag_ALL/Infer_Mask_T10_'
outdir10 = '/home/john/Documents/MESArthritis/Inference_Bone_SagLower_3/Mask_Infer_Sag_ALL/Inference_T10_'

test_list = [ f for f in  os.listdir(inputdir)]

### Inference of T6, T7, T8, T9, T10 Images Based on Mask R-CNN Model ###
for f in sorted (test_list):
    if f.endswith('.png'):
        try:
            segmask, output = segment_image.segmentImage(inputdir + f, extract_segmented_objects = False, mask_points_values = True, show_bboxes=False)
            class_list = segmask['class_ids'].tolist()
            class_str = ''.join(str(x) for x in class_list)
            a = []
            b = []
            c = []
            d = []
            e = []
            g = []
            h = []
            i = []
            j = []
            k = []

            if len(class_str) == 1:
                a = [ [individualArray] for individualArray in segmask['masks'] ]
            elif len(class_str) == 2:
                a,b = [ [individualArray] for individualArray in segmask['masks'] ]        
            elif len(class_str) == 3:
                a,b,c = [ [individualArray] for individualArray in segmask['masks'] ]
            elif len(class_str) == 4:
                a,b,c,d = [ [individualArray] for individualArray in segmask['masks'] ]        
            elif len(class_str) == 5:
                a,b,c,d,e = [ [individualArray] for individualArray in segmask['masks'] ]
            elif len(class_str) == 6:
                a,b,c,d,e,g = [ [individualArray] for individualArray in segmask['masks'] ]
            elif len(class_str) == 7:
                a,b,c,d,e,g,h = [ [individualArray] for individualArray in segmask['masks'] ]
            elif len(class_str) == 8:
                a,b,c,d,e,g,h,i = [ [individualArray] for individualArray in segmask['masks'] ]
            elif len(class_str) == 9:
                a,b,c,d,e,g,h,i,j = [ [individualArray] for individualArray in segmask['masks'] ]
            else:
                a,b,c,d,e,g,h,i,j,l = [ [individualArray] for individualArray in segmask['masks'] ]

            try:
                anest = [list(itertools.chain(*sub)) for sub in a]
                anest2 = [list(itertools.chain(*sub)) for sub in anest]
                astr = str(anest2)[2:-2]
                alist = astr.split(",")
                alist2 = [eval(i) for i in alist]
                adf = pd.DataFrame(alist2[1::2])
                min1 = int(adf.min())
            except:
                pass
            try:
                bnest = [list(itertools.chain(*sub)) for sub in b]
                bnest2 = [list(itertools.chain(*sub)) for sub in bnest]
                bstr = str(bnest2)[2:-2]
                blist = bstr.split(",")
                blist2 = [eval(i) for i in blist]
                bdf = pd.DataFrame(blist2[1::2])
                min2 = int(bdf.min())
            except:
                pass
            try:
                cnest = [list(itertools.chain(*sub)) for sub in c]
                cnest2 = [list(itertools.chain(*sub)) for sub in cnest]
                cstr = str(cnest2)[2:-2]
                clist = cstr.split(",")
                clist2 = [eval(i) for i in clist]
                cdf = pd.DataFrame(clist2[1::2])
                min3 = int(cdf.min())
            except:
                pass
            try:
                dnest = [list(itertools.chain(*sub)) for sub in d]
                dnest2 = [list(itertools.chain(*sub)) for sub in dnest]
                dstr = str(dnest2)[2:-2]
                dlist = dstr.split(",")
                dlist2 = [eval(i) for i in dlist]
                ddf = pd.DataFrame(dlist2[1::2])
                min4 = int(ddf.min())
            except:
                pass
            try:
                enest = [list(itertools.chain(*sub)) for sub in e]
                enest2 = [list(itertools.chain(*sub)) for sub in enest]
                estr = str(enest2)[2:-2]
                elist = estr.split(",")
                elist2 = [eval(i) for i in elist]
                edf = pd.DataFrame(elist2[1::2])
                min5 = int(edf.min())
            except:
                pass
            try:
                gnest = [list(itertools.chain(*sub)) for sub in g]
                gnest2 = [list(itertools.chain(*sub)) for sub in gnest]
                gstr = str(gnest2)[2:-2]
                glist = gstr.split(",")
                glist2 = [eval(i) for i in glist]
                gdf = pd.DataFrame(glist2[1::2])
                min6 = int(gdf.min())
            except:
                pass
            try:
                hnest = [list(itertools.chain(*sub)) for sub in h]
                hnest2 = [list(itertools.chain(*sub)) for sub in hnest]
                hstr = str(hnest2)[2:-2]
                hlist = hstr.split(",")
                hlist2 = [eval(i) for i in hlist]
                hdf = pd.DataFrame(hlist2[1::2])
                min7 = int(hdf.min())
            except:
                pass
            try:
                inest = [list(itertools.chain(*sub)) for sub in i]
                inest2 = [list(itertools.chain(*sub)) for sub in inest]
                istr = str(inest2)[2:-2]
                ilist = istr.split(",")
                ilist2 = [eval(i) for i in ilist]
                idf = pd.DataFrame(ilist2[1::2])
                min8 = int(idf.min())
            except:
                pass
            try:
                jnest = [list(itertools.chain(*sub)) for sub in j]
                jnest2 = [list(itertools.chain(*sub)) for sub in jnest]
                jstr = str(jnest2)[2:-2]
                jlist = jstr.split(",")
                jlist2 = [eval(i) for i in jlist]
                jdf = pd.DataFrame(jlist2[1::2])
                min9 = int(jdf.min())
            except:
                pass
            try:
                knest = [list(itertools.chain(*sub)) for sub in k]
                knest2 = [list(itertools.chain(*sub)) for sub in knest]
                kstr = str(knest2)[2:-2]
                klist = kstr.split(",")
                klist2 = [eval(i) for i in klist]
                kdf = pd.DataFrame(klist2[1::2])
                min10 = int(kdf.min())
            except:
                pass

            cv2.imwrite(outdir + f, output)

            myDict = {}
            if len(class_str) == 1:
                myDict[min1] = a
                a1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[0,1], dtype=object)
            elif len(class_str) == 2:
                myDict[min1] = a
                myDict[min2] = b
                a1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[0,1], dtype=object)
                b1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[1,1], dtype=object)
            elif len(class_str) == 3:
                myDict[min1] = a
                myDict[min2] = b
                myDict[min3] = c
                a1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[0,1], dtype=object)
                b1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[1,1], dtype=object)
                c1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[2,1], dtype=object)
            elif len(class_str) == 4:
                myDict[min1] = a
                myDict[min2] = b
                myDict[min3] = c
                myDict[min4] = d
                a1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[0,1], dtype=object)
                b1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[1,1], dtype=object)
                c1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[2,1], dtype=object)
                d1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[3,1], dtype=object)
            elif len(class_str) == 5:
                myDict[min1] = a
                myDict[min2] = b
                myDict[min3] = c
                myDict[min4] = d
                myDict[min5] = e
                a1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[0,1], dtype=object)
                b1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[1,1], dtype=object)
                c1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[2,1], dtype=object)
                d1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[3,1], dtype=object)
                e1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[4,1], dtype=object)
            elif len(class_str) == 6:
                myDict[min1] = a
                myDict[min2] = b
                myDict[min3] = c
                myDict[min4] = d
                myDict[min5] = e
                myDict[min6] = g
                a1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[1,1], dtype=object)
                b1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[2,1], dtype=object)
                c1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[3,1], dtype=object)
                d1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[4,1], dtype=object)
                e1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[5,1], dtype=object)
            elif len(class_str) == 7:
                myDict[min1] = a
                myDict[min2] = b
                myDict[min3] = c
                myDict[min4] = d
                myDict[min5] = e
                myDict[min6] = g
                myDict[min7] = h
                a1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[1,1], dtype=object)
                b1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[2,1], dtype=object)
                c1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[3,1], dtype=object)
                d1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[4,1], dtype=object)
                e1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[5,1], dtype=object)
            elif len(class_str) == 8:
                myDict[min1] = a
                myDict[min2] = b
                myDict[min3] = c
                myDict[min4] = d
                myDict[min5] = e
                myDict[min6] = g
                myDict[min7] = h
                myDict[min8] = i
                a1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[1,1], dtype=object)
                b1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[2,1], dtype=object)
                c1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[3,1], dtype=object)
                d1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[4,1], dtype=object)
                e1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[5,1], dtype=object)
            elif len(class_str) == 9:
                myDict[min1] = a
                myDict[min2] = b
                myDict[min3] = c
                myDict[min4] = d
                myDict[min5] = e
                myDict[min6] = g
                myDict[min7] = h
                myDict[min8] = i
                myDict[min9] = j
                a1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[1,1], dtype=object)
                b1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[2,1], dtype=object)
                c1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[3,1], dtype=object)
                d1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[4,1], dtype=object)
                e1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[5,1], dtype=object)
            else:
                myDict[min1] = a
                myDict[min2] = b
                myDict[min3] = c
                myDict[min4] = d
                myDict[min5] = e
                myDict[min6] = g
                myDict[min7] = h
                myDict[min8] = i
                myDict[min9] = j
                myDict[min10] = k
                a1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[1,1], dtype=object)
                b1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[2,1], dtype=object)
                c1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[3,1], dtype=object)
                d1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[4,1], dtype=object)
                e1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[5,1], dtype=object)

            try:
                for i in a1.tolist():
                    mask1 = np.asarray(Image.open(inputdir + f))
                    mask11 =  mask1[:,:,:3]
                    img1 = cv2.fillPoly(mask1, np.array(i), color=(0, 0, 0))
                    maskB = Image.fromarray(mask11)
                for i in a1.tolist():
                    mask2 = np.asarray(Image.open(inputdir + f))
                    mask22 =  mask2[:,:,:3]
                    img2 = cv2.fillPoly(mask2, np.array(i), color=(255, 255, 255))
                    maskW = Image.fromarray(mask22)
                    T6_Mask = np.subtract(maskW, maskB)
                    T6 = np.subtract(np.asarray(Image.open(inputdir + f))[:,:,:3], maskB)
                    cv2.imwrite(outdir1 + f, T6_Mask)
                    cv2.imwrite(outdir2 + f, T6)
                for i in b1.tolist():
                    mask3 = np.asarray(Image.open(inputdir + f))
                    mask33 =  mask3[:,:,:3]
                    img3 = cv2.fillPoly(mask3, np.array(i), color=(0, 0, 0))
                    maskB2 = Image.fromarray(mask33)
                for i in b1.tolist():
                    mask4 = np.asarray(Image.open(inputdir + f))
                    mask44 =  mask4[:,:,:3]
                    img4 = cv2.fillPoly(mask4, np.array(i), color=(255, 255, 255))
                    maskW2 = Image.fromarray(mask44)
                    T7_Mask = np.subtract(maskW2, maskB2)
                    T7 = np.subtract(np.asarray(Image.open(inputdir + f))[:,:,:3], maskB2)
                    cv2.imwrite(outdir3 + f, T7_Mask)
                    cv2.imwrite(outdir4 + f, T7)
                for i in c1.tolist():
                    mask5 = np.asarray(Image.open(inputdir + f))
                    mask55 =  mask5[:,:,:3]
                    img5 = cv2.fillPoly(mask5, np.array(i), color=(0, 0, 0))
                    maskB3 = Image.fromarray(mask55)
                for i in c1.tolist():
                    mask6 = np.asarray(Image.open(inputdir + f))
                    mask66 =  mask6[:,:,:3]
                    img6 = cv2.fillPoly(mask6, np.array(i), color=(255, 255, 255))
                    maskW3 = Image.fromarray(mask66)
                    T8_Mask = np.subtract(maskW3, maskB3)
                    T8 = np.subtract(np.asarray(Image.open(inputdir + f))[:,:,:3], maskB3)
                    cv2.imwrite(outdir5 + f, T8_Mask)
                    cv2.imwrite(outdir6 + f, T8)
                for i in d1.tolist():
                    mask7 = np.asarray(Image.open(inputdir + f))
                    mask77 =  mask7[:,:,:3]
                    img7 = cv2.fillPoly(mask7, np.array(i), color=(0, 0, 0))
                    maskB4 = Image.fromarray(mask77)
                for i in d1.tolist():
                    mask8 = np.asarray(Image.open(inputdir + f))
                    mask88 =  mask8[:,:,:3]
                    img8 = cv2.fillPoly(mask8, np.array(i), color=(255, 255, 255))
                    maskW4 = Image.fromarray(mask88)
                    T9_Mask = np.subtract(maskW4, maskB4)
                    T9 = np.subtract(np.asarray(Image.open(inputdir + f))[:,:,:3], maskB4)
                    cv2.imwrite(outdir7 + f, T9_Mask)
                    cv2.imwrite(outdir8 + f, T9)
                for i in e1.tolist():
                    mask9 = np.asarray(Image.open(inputdir + f))
                    mask99 =  mask9[:,:,:3]
                    img9 = cv2.fillPoly(mask9, np.array(i), color=(0, 0, 0))
                    maskB5 = Image.fromarray(mask99)
                for i in e1.tolist():
                    mask10 = np.asarray(Image.open(inputdir + f))
                    mask1010 =  mask10[:,:,:3]
                    img10 = cv2.fillPoly(mask10, np.array(i), color=(255, 255, 255))
                    maskW5 = Image.fromarray(mask1010)
                    T10_Mask = np.subtract(maskW5, maskB5)
                    T10 = np.subtract(np.asarray(Image.open(inputdir + f))[:,:,:3], maskB5)
                    cv2.imwrite(outdir9 + f, T10_Mask)
                    cv2.imwrite(outdir10 + f, T10)
            except:
                pass
        except:
            pass

In [ ]:
###Sagittal Upper###

In [ ]:
### Load Essential Packages ###
import numpy as np
from PIL import Image
import cv2
import os
import itertools
import pandas as pd
import operator

inputdir = '/home/john/Documents/MESArthritis/Inference_Bone_SagUpper/Infer_Sag_ALL/'
outdir = '/home/john/Documents/MESArthritis/Inference_Bone_SagUpper_3/Output_Sag_ALL/'
outdir1 = '/home/john/Documents/MESArthritis/Inference_Bone_SagUpper_3/Mask_Infer_Sag_ALL/Infer_Mask_T1_'
outdir2 = '/home/john/Documents/MESArthritis/Inference_Bone_SagUpper_3/Mask_Infer_Sag_ALL/Inference_T1_'
outdir3 = '/home/john/Documents/MESArthritis/Inference_Bone_SagUpper_3/Mask_Infer_Sag_ALL/Infer_Mask_T2_'
outdir4 = '/home/john/Documents/MESArthritis/Inference_Bone_SagUpper_3/Mask_Infer_Sag_ALL/Inference_T2_'
outdir5 = '/home/john/Documents/MESArthritis/Inference_Bone_SagUpper_3/Mask_Infer_Sag_ALL/Infer_Mask_T3_'
outdir6 = '/home/john/Documents/MESArthritis/Inference_Bone_SagUpper_3/Mask_Infer_Sag_ALL/Inference_T3_'
outdir7 = '/home/john/Documents/MESArthritis/Inference_Bone_SagUpper_3/Mask_Infer_Sag_ALL/Infer_Mask_T4_'
outdir8 = '/home/john/Documents/MESArthritis/Inference_Bone_SagUpper_3/Mask_Infer_Sag_ALL/Inference_T4_'
outdir9 = '/home/john/Documents/MESArthritis/Inference_Bone_SagUpper_3/Mask_Infer_Sag_ALL/Infer_Mask_T5_'
outdir10 = '/home/john/Documents/MESArthritis/Inference_Bone_SagUpper_3/Mask_Infer_Sag_ALL/Inference_T5_'

test_list = [ f for f in  os.listdir(inputdir)]

### Inference of T1, T2, T3, T4, T5 Images Based on Mask R-CNN Model ###
for f in sorted (test_list):
    if f.endswith('.png'):
        try:
            segmask, output = segment_image.segmentImage(inputdir + f, extract_segmented_objects = False, mask_points_values = True, show_bboxes=False)
            class_list = segmask['class_ids'].tolist()
            class_str = ''.join(str(x) for x in class_list)
            a = []
            b = []
            c = []
            d = []
            e = []
            g = []
            h = []
            i = []
            j = []
            k = []

            if len(class_str) == 1:
                a = [ [individualArray] for individualArray in segmask['masks'] ]
            elif len(class_str) == 2:
                a,b = [ [individualArray] for individualArray in segmask['masks'] ]        
            elif len(class_str) == 3:
                a,b,c = [ [individualArray] for individualArray in segmask['masks'] ]
            elif len(class_str) == 4:
                a,b,c,d = [ [individualArray] for individualArray in segmask['masks'] ]        
            elif len(class_str) == 5:
                a,b,c,d,e = [ [individualArray] for individualArray in segmask['masks'] ]
            elif len(class_str) == 6:
                a,b,c,d,e,g = [ [individualArray] for individualArray in segmask['masks'] ]
            elif len(class_str) == 7:
                a,b,c,d,e,g,h = [ [individualArray] for individualArray in segmask['masks'] ]
            elif len(class_str) == 8:
                a,b,c,d,e,g,h,i = [ [individualArray] for individualArray in segmask['masks'] ]
            elif len(class_str) == 9:
                a,b,c,d,e,g,h,i,j = [ [individualArray] for individualArray in segmask['masks'] ]
            else:
                a,b,c,d,e,g,h,i,j,l = [ [individualArray] for individualArray in segmask['masks'] ]

            try:
                anest = [list(itertools.chain(*sub)) for sub in a]
                anest2 = [list(itertools.chain(*sub)) for sub in anest]
                astr = str(anest2)[2:-2]
                alist = astr.split(",")
                alist2 = [eval(i) for i in alist]
                adf = pd.DataFrame(alist2[1::2])
                min1 = int(adf.min())
            except:
                pass
            try:
                bnest = [list(itertools.chain(*sub)) for sub in b]
                bnest2 = [list(itertools.chain(*sub)) for sub in bnest]
                bstr = str(bnest2)[2:-2]
                blist = bstr.split(",")
                blist2 = [eval(i) for i in blist]
                bdf = pd.DataFrame(blist2[1::2])
                min2 = int(bdf.min())
            except:
                pass
            try:
                cnest = [list(itertools.chain(*sub)) for sub in c]
                cnest2 = [list(itertools.chain(*sub)) for sub in cnest]
                cstr = str(cnest2)[2:-2]
                clist = cstr.split(",")
                clist2 = [eval(i) for i in clist]
                cdf = pd.DataFrame(clist2[1::2])
                min3 = int(cdf.min())
            except:
                pass
            try:
                dnest = [list(itertools.chain(*sub)) for sub in d]
                dnest2 = [list(itertools.chain(*sub)) for sub in dnest]
                dstr = str(dnest2)[2:-2]
                dlist = dstr.split(",")
                dlist2 = [eval(i) for i in dlist]
                ddf = pd.DataFrame(dlist2[1::2])
                min4 = int(ddf.min())
            except:
                pass
            try:
                enest = [list(itertools.chain(*sub)) for sub in e]
                enest2 = [list(itertools.chain(*sub)) for sub in enest]
                estr = str(enest2)[2:-2]
                elist = estr.split(",")
                elist2 = [eval(i) for i in elist]
                edf = pd.DataFrame(elist2[1::2])
                min5 = int(edf.min())
            except:
                pass
            try:
                gnest = [list(itertools.chain(*sub)) for sub in g]
                gnest2 = [list(itertools.chain(*sub)) for sub in gnest]
                gstr = str(gnest2)[2:-2]
                glist = gstr.split(",")
                glist2 = [eval(i) for i in glist]
                gdf = pd.DataFrame(glist2[1::2])
                min6 = int(gdf.min())
            except:
                pass
            try:
                hnest = [list(itertools.chain(*sub)) for sub in h]
                hnest2 = [list(itertools.chain(*sub)) for sub in hnest]
                hstr = str(hnest2)[2:-2]
                hlist = hstr.split(",")
                hlist2 = [eval(i) for i in hlist]
                hdf = pd.DataFrame(hlist2[1::2])
                min7 = int(hdf.min())
            except:
                pass
            try:
                inest = [list(itertools.chain(*sub)) for sub in i]
                inest2 = [list(itertools.chain(*sub)) for sub in inest]
                istr = str(inest2)[2:-2]
                ilist = istr.split(",")
                ilist2 = [eval(i) for i in ilist]
                idf = pd.DataFrame(ilist2[1::2])
                min8 = int(idf.min())
            except:
                pass
            try:
                jnest = [list(itertools.chain(*sub)) for sub in j]
                jnest2 = [list(itertools.chain(*sub)) for sub in jnest]
                jstr = str(jnest2)[2:-2]
                jlist = jstr.split(",")
                jlist2 = [eval(i) for i in jlist]
                jdf = pd.DataFrame(jlist2[1::2])
                min9 = int(jdf.min())
            except:
                pass
            try:
                knest = [list(itertools.chain(*sub)) for sub in k]
                knest2 = [list(itertools.chain(*sub)) for sub in knest]
                kstr = str(knest2)[2:-2]
                klist = kstr.split(",")
                klist2 = [eval(i) for i in klist]
                kdf = pd.DataFrame(klist2[1::2])
                min10 = int(kdf.min())
            except:
                pass

            cv2.imwrite(outdir + f, output)

            myDict = {}
            if len(class_str) == 1:
                myDict[min1] = a
                e1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[0,1], dtype=object)
            elif len(class_str) == 2:
                myDict[min1] = a
                myDict[min2] = b
                d1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[0,1], dtype=object)
                e1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[1,1], dtype=object)
            elif len(class_str) == 3:
                myDict[min1] = a
                myDict[min2] = b
                myDict[min3] = c
                c1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[0,1], dtype=object)
                d1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[1,1], dtype=object)
                e1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[2,1], dtype=object)
            elif len(class_str) == 4:
                myDict[min1] = a
                myDict[min2] = b
                myDict[min3] = c
                myDict[min4] = d
                b1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[0,1], dtype=object)
                c1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[1,1], dtype=object)
                d1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[2,1], dtype=object)
                e1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[3,1], dtype=object)
            elif len(class_str) == 5:
                myDict[min1] = a
                myDict[min2] = b
                myDict[min3] = c
                myDict[min4] = d
                myDict[min5] = e
                a1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[0,1], dtype=object)
                b1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[1,1], dtype=object)
                c1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[2,1], dtype=object)
                d1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[3,1], dtype=object)
                e1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[4,1], dtype=object)
            elif len(class_str) == 6:
                myDict[min1] = a
                myDict[min2] = b
                myDict[min3] = c
                myDict[min4] = d
                myDict[min5] = e
                myDict[min6] = g
                a1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[1,1], dtype=object)
                b1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[2,1], dtype=object)
                c1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[3,1], dtype=object)
                d1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[4,1], dtype=object)
                e1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[5,1], dtype=object)
            elif len(class_str) == 7:
                myDict[min1] = a
                myDict[min2] = b
                myDict[min3] = c
                myDict[min4] = d
                myDict[min5] = e
                myDict[min6] = g
                myDict[min7] = h
                a1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[1,1], dtype=object)
                b1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[2,1], dtype=object)
                c1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[3,1], dtype=object)
                d1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[4,1], dtype=object)
                e1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[5,1], dtype=object)
            elif len(class_str) == 8:
                myDict[min1] = a
                myDict[min2] = b
                myDict[min3] = c
                myDict[min4] = d
                myDict[min5] = e
                myDict[min6] = g
                myDict[min7] = h
                myDict[min8] = i
                a1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[1,1], dtype=object)
                b1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[2,1], dtype=object)
                c1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[3,1], dtype=object)
                d1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[4,1], dtype=object)
                e1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[5,1], dtype=object)
            elif len(class_str) == 9:
                myDict[min1] = a
                myDict[min2] = b
                myDict[min3] = c
                myDict[min4] = d
                myDict[min5] = e
                myDict[min6] = g
                myDict[min7] = h
                myDict[min8] = i
                myDict[min9] = j
                a1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[1,1], dtype=object)
                b1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[2,1], dtype=object)
                c1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[3,1], dtype=object)
                d1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[4,1], dtype=object)
                e1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[5,1], dtype=object)
            else:
                myDict[min1] = a
                myDict[min2] = b
                myDict[min3] = c
                myDict[min4] = d
                myDict[min5] = e
                myDict[min6] = g
                myDict[min7] = h
                myDict[min8] = i
                myDict[min9] = j
                myDict[min10] = k
                a1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[1,1], dtype=object)
                b1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[2,1], dtype=object)
                c1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[3,1], dtype=object)
                d1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[4,1], dtype=object)
                e1 = np.array((np.array((sorted(myDict.items(), key=operator.itemgetter(0))), dtype=object))[5,1], dtype=object)

            try:
                for i in a1.tolist():
                    mask1 = np.asarray(Image.open(inputdir + f))
                    mask11 =  mask1[:,:,:3]
                    img1 = cv2.fillPoly(mask1, np.array(i), color=(0, 0, 0))
                    maskB = Image.fromarray(mask11)
                for i in a1.tolist():
                    mask2 = np.asarray(Image.open(inputdir + f))
                    mask22 =  mask2[:,:,:3]
                    img2 = cv2.fillPoly(mask2, np.array(i), color=(255, 255, 255))
                    maskW = Image.fromarray(mask22)
                    T1_Mask = np.subtract(maskW, maskB)
                    T1 = np.subtract(np.asarray(Image.open(inputdir + f))[:,:,:3], maskB)
                    cv2.imwrite(outdir1 + f, T1_Mask)
                    cv2.imwrite(outdir2 + f, T1)
                for i in b1.tolist():
                    mask3 = np.asarray(Image.open(inputdir + f))
                    mask33 =  mask3[:,:,:3]
                    img3 = cv2.fillPoly(mask3, np.array(i), color=(0, 0, 0))
                    maskB2 = Image.fromarray(mask33)
                for i in b1.tolist():
                    mask4 = np.asarray(Image.open(inputdir + f))
                    mask44 =  mask4[:,:,:3]
                    img4 = cv2.fillPoly(mask4, np.array(i), color=(255, 255, 255))
                    maskW2 = Image.fromarray(mask44)
                    T2_Mask = np.subtract(maskW2, maskB2)
                    T2 = np.subtract(np.asarray(Image.open(inputdir + f))[:,:,:3], maskB2)
                    cv2.imwrite(outdir3 + f, T2_Mask)
                    cv2.imwrite(outdir4 + f, T2)
                for i in c1.tolist():
                    mask5 = np.asarray(Image.open(inputdir + f))
                    mask55 =  mask5[:,:,:3]
                    img5 = cv2.fillPoly(mask5, np.array(i), color=(0, 0, 0))
                    maskB3 = Image.fromarray(mask55)
                for i in c1.tolist():
                    mask6 = np.asarray(Image.open(inputdir + f))
                    mask66 =  mask6[:,:,:3]
                    img6 = cv2.fillPoly(mask6, np.array(i), color=(255, 255, 255))
                    maskW3 = Image.fromarray(mask66)
                    T3_Mask = np.subtract(maskW3, maskB3)
                    T3 = np.subtract(np.asarray(Image.open(inputdir + f))[:,:,:3], maskB3)
                    cv2.imwrite(outdir5 + f, T3_Mask)
                    cv2.imwrite(outdir6 + f, T3)
                for i in d1.tolist():
                    mask7 = np.asarray(Image.open(inputdir + f))
                    mask77 =  mask7[:,:,:3]
                    img7 = cv2.fillPoly(mask7, np.array(i), color=(0, 0, 0))
                    maskB4 = Image.fromarray(mask77)
                for i in d1.tolist():
                    mask8 = np.asarray(Image.open(inputdir + f))
                    mask88 =  mask8[:,:,:3]
                    img8 = cv2.fillPoly(mask8, np.array(i), color=(255, 255, 255))
                    maskW4 = Image.fromarray(mask88)
                    T4_Mask = np.subtract(maskW4, maskB4)
                    T4 = np.subtract(np.asarray(Image.open(inputdir + f))[:,:,:3], maskB4)
                    cv2.imwrite(outdir7 + f, T4_Mask)
                    cv2.imwrite(outdir8 + f, T4)
                for i in e1.tolist():
                    mask9 = np.asarray(Image.open(inputdir + f))
                    mask99 =  mask9[:,:,:3]
                    img9 = cv2.fillPoly(mask9, np.array(i), color=(0, 0, 0))
                    maskB5 = Image.fromarray(mask99)
                for i in e1.tolist():
                    mask10 = np.asarray(Image.open(inputdir + f))
                    mask1010 =  mask10[:,:,:3]
                    img10 = cv2.fillPoly(mask10, np.array(i), color=(255, 255, 255))
                    maskW5 = Image.fromarray(mask1010)
                    T5_Mask = np.subtract(maskW5, maskB5)
                    T5 = np.subtract(np.asarray(Image.open(inputdir + f))[:,:,:3], maskB5)
                    cv2.imwrite(outdir9 + f, T5_Mask)
                    cv2.imwrite(outdir10 + f, T5)
            except:
                pass
        except:
            pass

In [ ]:
from __future__ import print_function

import collections
import csv
import logging

import SimpleITK as sitk

import radiomics
from radiomics import featureextractor
import pydicom
import pylab
from pydicom import dcmread

from numpy import asarray
import numpy as np
import pandas as pd
import nrrd

import os
from os import path, rename

def main():
  outPath = "/home/john/Documents/MESArthritis/Inference_Bone_2/Radiomics/"

  inputCSV = os.path.join(outPath, 'File_Location_vBMD.csv')
  outputFilepath = os.path.join(outPath, 'Radiomics_Features_vBMD.csv')
  progress_filename = os.path.join(outPath, 'Pyradiomics_Log_vBMD.txt')
  params = os.path.join(outPath, './', 'Params_vBMD.yaml')

  # Configure logging
  rLogger = logging.getLogger('radiomics')

  # Create handler for writing to log file
  handler = logging.FileHandler(filename=progress_filename, mode='w')
  handler.setFormatter(logging.Formatter('%(levelname)s:%(name)s: %(message)s'))
  rLogger.addHandler(handler)

  # Initialize logging for batch log messages
  logger = rLogger.getChild('batch')

  # Set verbosity level for output to stderr (level = ERROR)
  radiomics.setVerbosity(40)

  logger.info('pyradiomics version: %s', radiomics.__version__)
  logger.info('Loading CSV')

  flists = []
  try:
    with open(inputCSV, 'r') as inFile:
      cr = csv.DictReader(inFile, lineterminator='\n')
      flists = [row for row in cr]
  except Exception:
    logger.error('CSV READ FAILED', exc_info=True)

  logger.info('Loading Done')
  logger.info('Patients: %d', len(flists))

  if os.path.isfile(params):
    extractor = featureextractor.RadiomicsFeatureExtractor(params)
  else:  # Parameter file not found, use hardcoded settings instead
    settings = {}
    settings['binWidth'] = 25
    settings['resampledPixelSpacing'] = None  # [3,3,3]
    settings['interpolator'] = sitk.sitkBSpline
    settings['enableCExtensions'] = True

    extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
    # extractor.enableInputImages(wavelet= {'level': 2})

  logger.info('Enabled input images types: %s', extractor.enabledImagetypes)
  logger.info('Enabled features: %s', extractor.enabledFeatures)
  logger.info('Current settings: %s', extractor.settings)

  headers = None

  for idx, entry in enumerate(flists, start=1):

    logger.info("(%d/%d) Processing Patient (Image: %s, Mask: %s)", idx, len(flists), entry['Image'], entry['Mask'])

    imageFilepath = entry['Image']
    maskFilepath = entry['Mask']
    label = entry['Label']

    if (imageFilepath is not None) and (maskFilepath is not None):
      featureVector = collections.OrderedDict(entry)
      featureVector['Image'] = os.path.basename(imageFilepath)
      featureVector['Mask'] = os.path.basename(maskFilepath)

      try:
        featureVector.update(extractor.execute(sitk.ReadImage(imageFilepath,sitk.sitkUInt8), sitk.ReadImage(maskFilepath,sitk.sitkUInt8), label=255))

        with open(outputFilepath, 'a') as outputFile:
          writer = csv.writer(outputFile, lineterminator='\n')
          if headers is None:
            headers = list(featureVector.keys())
            writer.writerow(headers)

          row = []
          for h in headers:
            row.append(featureVector.get(h, "N/A"))
          writer.writerow(row)
      except Exception:
        logger.error('FEATURE EXTRACTION FAILED', exc_info=True)

if __name__ == '__main__':
  main()

In [ ]:
### Load Essential Packages ###
import cv2
import os
import numpy as np
import glob
from glob import iglob
import shutil

for filename in sorted(glob.glob('/home/john/Documents/MESArthritis/Inference_Bone/Binary_ROI3/*Infer_Mask_*.png')):
    path, filenames = os.path.split(filename)
    path2 = os.path.basename(path)
    try:
        shutil.copyfile(filename, '/home/john/Documents/MESArthritis/Ground_Truth_Bone/test/' + filenames[11:22] + '/masks/' + filenames)
    except Exception:
        pass

In [ ]:
### Load Essential Packages ###
import cv2
import os
import numpy as np
import glob
from glob import iglob
import shutil

for filename in sorted(glob.glob('/home/john/Documents/MESArthritis/Inference_Bone_SagLower_3/Mask_Infer_Sag_ALL/Infer_Mask_*.png')):
    path, filenames = os.path.split(filename)
    path2 = os.path.basename(path)
    try:
        shutil.copyfile(filename, '/home/john/Documents/MESArthritis/Ground_Truth_Bone_SagLower_C2/test/' + filenames[14:25] + '/masks/' + filenames)
    except Exception:
        pass

In [ ]:
### Load Essential Packages ###
import cv2
import os
import numpy as np
import glob
from glob import iglob
import shutil

for filename in sorted(glob.glob('/home/john/Documents/MESArthritis/Inference_Bone_SagUpper_3/Mask_Infer_Sag_ALL/Infer_Mask_*.png')):
    path, filenames = os.path.split(filename)
    path2 = os.path.basename(path)
    try:
        shutil.copyfile(filename, '/home/john/Documents/MESArthritis/Ground_Truth_Bone_SagUpper_C2/test/' + filenames[14:25] + '/masks/' + filenames)
    except Exception:
        pass

In [ ]:
from PIL import Image
import SimpleITK as sitk
import cv2
import os
import numpy as np
import glob
from glob import iglob
import csv

for file1 in sorted(glob.glob('/home/john/Documents/MESArthritis/Ground_Truth_Bone_2/test/**/**/*_ROI.png')):
    for file2 in sorted(glob.glob('/home/john/Documents/MESArthritis/Ground_Truth_Bone_2/test/**/**/Infer_Mask_*.png')):
        if file1[78:93] == file2[89:104]:
            path, filenames = os.path.split(file1)
            path2 = os.path.basename(path)
            img = Image.open(file1)
            img2 = Image.open(file2).convert("L")
            img = np.array(img)
            img2 = np.array(img2)
            intersection = np.logical_and(img, img2)
            union = np.logical_or(img, img2)
            iou_score = np.sum(intersection) / np.sum(union)
            dice_score = (2*np.sum(intersection))/(np.sum(union)+np.sum(intersection))
            mean_pixel_accuracy = 1.0 * np.sum(img) / (np.spacing(1) + np.sum(img2))
            fields=[filenames[0:15], "ROI_Spine", iou_score, dice_score, mean_pixel_accuracy]
    with open('/home/john/Documents/MESArthritis/Inference_Bone_2/Metrics_Test_Axial_Whole.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(fields)

In [ ]:
from PIL import Image, ImageChops
import glob
from glob import iglob
import csv
import os

for image_one in sorted(glob.glob('/home/john/Documents/MESArthritis/Ground_Truth_Bone_SagLower_C2/train/**/**/*_T6_*.png')):
    for image_two in sorted(glob.glob('/home/john/Documents/MESArthritis/Ground_Truth_Bone_SagLower_C2/train/**/**/*_T7_*.png')):
        if image_one[69:80] == image_two[69:80]:
            for image_three in sorted(glob.glob('/home/john/Documents/MESArthritis/Ground_Truth_Bone_SagLower_C2/train/**/**/*_T8_*.png')):
                if image_one[69:80] == image_two[69:80] and image_one[69:80] == image_three[69:80]:
                    for image_four in sorted(glob.glob('/home/john/Documents/MESArthritis/Ground_Truth_Bone_SagLower_C2/train/**/**/*_T9_*.png')):
                        if image_one[69:80] == image_two[69:80] and image_one[69:80] == image_three[69:80] and image_one[69:80] == image_four[69:80]:
                            for image_five in sorted(glob.glob('/home/john/Documents/MESArthritis/Ground_Truth_Bone_SagLower_C2/train/**/**/*_T10_*.png')):
                                if image_one[69:80] == image_two[69:80] and image_one[69:80] == image_three[69:80] and image_one[69:80] == image_four[69:80] and image_one[69:80] == image_five[69:80]:
                                    image_one_img = Image.open(image_one)
                                    image_two_img = Image.open(image_two)
                                    image_three_img = Image.open(image_three)
                                    image_four_img = Image.open(image_four)
                                    image_five_img = Image.open(image_five)
                                    path, filenames = os.path.split(image_one)
                                    out = ImageChops.add(image_one_img, image_two_img)
                                    out2 = ImageChops.add(out, image_three_img)
                                    out3 = ImageChops.add(out2, image_four_img)
                                    out4 = ImageChops.add(out3, image_five_img)
                                    out4.save(path + "/" + path[69:80] + "_Predicted.png")

In [ ]:
from PIL import Image, ImageChops
import glob
from glob import iglob
import csv
import os

for image_one in sorted(glob.glob('/home/john/Documents/MESArthritis/Ground_Truth_Bone_SagUpper_C2/train/**/**/*_T1_*.png')):
    for image_two in sorted(glob.glob('/home/john/Documents/MESArthritis/Ground_Truth_Bone_SagUpper_C2/train/**/**/*_T2_*.png')):
        if image_one[69:80] == image_two[69:80]:
            for image_three in sorted(glob.glob('/home/john/Documents/MESArthritis/Ground_Truth_Bone_SagUpper_C2/train/**/**/*_T3_*.png')):
                if image_one[69:80] == image_two[69:80] and image_one[69:80] == image_three[69:80]:
                    for image_four in sorted(glob.glob('/home/john/Documents/MESArthritis/Ground_Truth_Bone_SagUpper_C2/train/**/**/*_T4_*.png')):
                        if image_one[69:80] == image_two[69:80] and image_one[69:80] == image_three[69:80] and image_one[69:80] == image_four[69:80]:
                            for image_five in sorted(glob.glob('/home/john/Documents/MESArthritis/Ground_Truth_Bone_SagUpper_C2/train/**/**/*_T5_*.png')):
                                if image_one[69:80] == image_two[69:80] and image_one[69:80] == image_three[69:80] and image_one[69:80] == image_four[69:80] and image_one[69:80] == image_five[69:80]:
                                    image_one_img = Image.open(image_one)
                                    image_two_img = Image.open(image_two)
                                    image_three_img = Image.open(image_three)
                                    image_four_img = Image.open(image_four)
                                    image_five_img = Image.open(image_five)
                                    path, filenames = os.path.split(image_one)
                                    out = ImageChops.add(image_one_img, image_two_img)
                                    out2 = ImageChops.add(out, image_three_img)
                                    out3 = ImageChops.add(out2, image_four_img)
                                    out4 = ImageChops.add(out3, image_five_img)
                                    out4.save(path + "/" + path[69:80] + "_Predicted.png")

In [ ]:
import json
import glob
from glob import iglob
import os

for jsonfile in sorted(glob.glob('/home/john/Documents/MESArthritis/Coco_Anno/*.json')):
    with open(jsonfile, 'r+') as f:
        path, filenames = os.path.split(jsonfile)
        data = json.load(f)
        data['imagePath'] = filenames[:-5] + ".png" # <--- add `id` value.
        f.seek(0)        # <--- should reset file position to the beginning.
        json.dump(data, f, indent=4)
        f.truncate()     # remove remaining part